In [15]:
import requests
from bs4 import BeautifulSoup
from collections import Counter
import pandas as pd
import time
import random
import re
import os

In [16]:
def get_time_seq(co_id, year):

    url = 'https://mops.twse.com.tw/mops/web/t05st01'

    params = {
    'encodeURIComponent': '1',
    'step': '1',
    'firstin': '1',
    'off': '1',
    'queryName': 'co_id',
    'inpuType': 'co_id',
    'TYPEK': 'all',
    'isnew': 'false',
    'co_id': co_id,
    'year' : year
    }

    for i in range(20):
        try:
            response = requests.post(url, data=params, timeout = 10)
            response.encoding = 'utf-8'
            html_doc = response.text
            soup = BeautifulSoup(html_doc, 'html.parser')
            
            alarm = soup.find('td')
            if('Overrun' in alarm.text):
                now = time.strftime("%Y-%m-%d %H:%M:%S")
                print("連線被阻擋RRRRRRRRRRRRRRRRRRRRRR(" + now + ")，兩分鐘後重新連線")
                time.sleep(120)
                continue


            table = soup.find('table', {'class': 'hasBorder'})
            rows = table.findAll('tr')

            button = soup.find('td')


            button_value = button.find_all('input', value = "詳細資料")
            break
        
        except TimeoutError:
            now = time.strftime("%Y-%m-%d %H:%M:%S")
            print("連線被中斷(" + now + ")，兩分鐘後重新連線")
            time.sleep(120)
            continue

        except requests.exceptions.ReadTimeout:
            now = time.strftime("%Y-%m-%d %H:%M:%S")
            print("連線被中斷(" + now + ")，兩分鐘後重新連線")
            time.sleep(120)
            continue

        except AttributeError:
            return [], [], []
        
  

    datetime_list = [] #格式範例: 110/01/01 09:00:00
    date_list = []  #格式範例: 110/01/01
    seq_no_list = [] #格式範例: ['1','2','3'...]

    for row in rows:
        cols = row.findAll('td')
        if len(cols) >= 3:
            datetime_list.append(cols[2].text.strip() + " " +cols[3].text.strip())
            date_list.append(cols[2].text.strip())

    for value in button_value:
        matche = re.findall(r"document.t05st01_fm.seq_no.value='(\d+)", value.get('onclick'))
        seq_no_list.append(matche[0])


    return datetime_list, date_list, seq_no_list

# a,b,c = get_time_seq('2330', '110')
# print((a))
# print(b)
# print((c))

In [17]:
#處理時間格式的function(送入datetime，例如110/01/01 09:00:00)

#回傳民國年('110')
def get_yyy(datetime): 
    return datetime.split('/')[0]


#回傳年月日('20210101')
def get_yyyymmdd(datetime):
    return str(int(datetime.split("/")[0])+1911) + datetime.split("/")[1] + datetime.split("/")[2].split(" ")[0]

#回傳時分秒('090000')
def get_hhmmss(datetime):
    return datetime.split("/")[2].split(" ")[1].replace(":", "")

In [18]:
def output_daily_info(co_id, month, datetime, seq_no):

    yyyymmdd = get_yyyymmdd(datetime)
    hhmmss = get_hhmmss(datetime)
    y = get_yyy(datetime)

    params = {
    "firstin": "true",
    "b_date": "",
    "e_date": "",
    "TYPEK": "all",
    "year": str(y),  #格式為民國年
    "month": "all",
    "type": "",
    "co_id": str(co_id), #公司代號
    "spoke_date": str(yyyymmdd), #格式為yyyymmdd
    "spoke_time": str(hhmmss), #格式為hhmmss
    "seq_no": str(seq_no), #當日的重大訊息id(由1開始，每日更新)
    "MEETING_STEP":"",
    "e_month": "all",
    "step": "2",
    "off": "1",
    "month" : str(month)
    }

    url = 'https://mops.twse.com.tw/mops/web/t05st01'

    for i in range(30):
        try:
            time.sleep(random.randint(1, 3))
            response = requests.post(url, data=params, timeout=10)
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text, 'html.parser')

            alarm = soup.find('td')
            if('Overrun' in alarm.text):
                now = time.strftime("%Y-%m-%d %H:%M:%S")
                print("連線被阻擋RRRRRRRRRRRRRRRRRRRRRR(" + now + ")，兩分鐘後重新連線")
                time.sleep(120)
                continue

            contents = soup.find_all('td', {'class': 'odd'})
            break
        except requests.exceptions.ReadTimeout:
            now = time.strftime("%Y-%m-%d %H:%M:%S")
            print("連線被中斷(" + now + ")，兩分鐘後重新連線")
            time.sleep(120)
            continue

        except TimeoutError:
            now = time.strftime("%Y-%m-%d %H:%M:%S")
            print("連線被中斷(" + now + ")，兩分鐘後重新連線")
            time.sleep(120)
            continue

    soup = BeautifulSoup(response.text, 'html.parser')
    contents = soup.find_all('td', {'class': 'odd'})

    header = ['序號',
            '發言日期',
            '發言時間',
            '發言人',
            '發言人職稱',
            '發言人電話',
            '主旨',
            '符合條款',
            '事實發生日',
            '說明']
    content_list = []
    for content in contents:
        content_list.append(content.text.strip().replace('\n', ' ').replace('\r', '').replace('\t', '').replace('\xa0', ''))

    #將內容存為字典
    dic = dict(zip(header, content_list))
    
    return dic

In [19]:
def crawer(co_id, year, month):
    print("開始蒐集"+ str(co_id)+ "公司" +str(year)+"年"+str(month)+"月的重要資訊")

    #建立要輸出的dataframe
    df_header = ['序號',
                '發言日期',
                '發言時間',
                '發言人',
                '發言人職稱',
                '發言人電話',
                '主旨',
                '符合條款',
                '事實發生日',
                '說明']

    df = pd.DataFrame(columns=df_header)

    datetime_list, date_list, seqno_list = get_time_seq(co_id, year)    
    
    if(datetime_list == []):
        return df

    for i in range(len(datetime_list)):
            
        if(datetime_list[i].split(" ")[0].split("/")[1] == month):
            day = datetime_list[i].split(" ")[0]
            
            info = output_daily_info(co_id, month, datetime_list[i], seqno_list[i])

            print("已蒐集到" + day + "的第 " + str(seqno_list[i]) + " 序號資料( " + time.strftime("%Y-%m-%d %H:%M:%S") + " )")

            df = pd.concat([df, pd.DataFrame(info, index=[0])], ignore_index=True)

            time.sleep(random.randint(1, 3))

    df['公司代號'] = co_id
    return df

In [20]:
def new_folder(co_id, year):
    folder = os.path.exists('C:\\Users\\user\\Desktop\\Crawler\\data\\'+ co_id )
    if not folder:
        os.makedirs('C:\\Users\\user\\Desktop\\Crawler\\data\\'+ co_id)
    else:
        return

In [21]:
#主程式
if __name__ == "__main__":
    
    month_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']   #月份
    co_id_list = ['6452', '3383', '1218', '2330', '2454', '2912', '1712', '1234', '2891', '2633'] #公司代號
    year_list = ['110','111'] #年度

    for co_id in co_id_list:
        for year in year_list:
            for month in month_list:
                    df = crawer(co_id, year, month)
                    print("該公司(" + co_id + ")" + year + "年" + month + "月，蒐集到" + str(len(df)) + "筆重要資訊")
                    if(df.empty == False):
                        new_folder(co_id, year)
                        df.to_csv('C:\\Users\\user\\Desktop\\Crawler\\data\\'+ co_id +"\\" + co_id +"-"+ year + month + '.csv', encoding='utf-8-sig')
        print("============="+str(co_id)+"公司蒐集完畢==============")

開始蒐集6452公司110年01月的重要資訊
該公司(6452)110年01月，蒐集到0筆重要資訊
開始蒐集6452公司110年02月的重要資訊
已蒐集到110/02/23的第 1 序號資料( 2023-03-23 23:26:12 )
已蒐集到110/02/26的第 4 序號資料( 2023-03-23 23:26:17 )
已蒐集到110/02/26的第 1 序號資料( 2023-03-23 23:26:20 )
該公司(6452)110年02月，蒐集到3筆重要資訊
開始蒐集6452公司110年03月的重要資訊
已蒐集到110/03/08的第 1 序號資料( 2023-03-23 23:26:25 )
已蒐集到110/03/31的第 1 序號資料( 2023-03-23 23:26:29 )
該公司(6452)110年03月，蒐集到2筆重要資訊
開始蒐集6452公司110年04月的重要資訊
該公司(6452)110年04月，蒐集到0筆重要資訊
開始蒐集6452公司110年05月的重要資訊
已蒐集到110/05/05的第 1 序號資料( 2023-03-23 23:26:31 )
已蒐集到110/05/05的第 3 序號資料( 2023-03-23 23:26:34 )
已蒐集到110/05/21的第 1 序號資料( 2023-03-23 23:26:38 )
已蒐集到110/05/27的第 1 序號資料( 2023-03-23 23:26:43 )
該公司(6452)110年05月，蒐集到4筆重要資訊
開始蒐集6452公司110年06月的重要資訊
該公司(6452)110年06月，蒐集到0筆重要資訊
開始蒐集6452公司110年07月的重要資訊
已蒐集到110/07/01的第 1 序號資料( 2023-03-23 23:26:47 )
已蒐集到110/07/16的第 1 序號資料( 2023-03-23 23:26:50 )
該公司(6452)110年07月，蒐集到2筆重要資訊
開始蒐集6452公司110年08月的重要資訊
該公司(6452)110年08月，蒐集到0筆重要資訊
開始蒐集6452公司110年09月的重要資訊
該公司(6452)110年09月，蒐集到0筆重要資訊
開始蒐集6452公司110年10月的重要資訊
該公司(6452)110年10月，蒐集到0

KeyboardInterrupt: 